In [91]:
%load_ext autoreload
%autoreload 2

from toolz import take
import torch as th
from torch.utils.data import DataLoader
from torch import tensor, optim, nn
from torch.nn import functional as F

from wxml.model import MLP
from wxml.hamming import make_data_hamming, wt
from wxml.data import make_splits
from wxml.train import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
xs, ys = make_data_hamming(100000)

[print(xy) for xy in take(5, zip(xs, ys))]

ds_train, ds_val, ds_test = make_splits(xs, ys)
train_loader, val_loader, test_loader = [DataLoader(d, batch_size=32, shuffle=False) for d in [ds_train, ds_val, ds_test]]

(tensor([0., 0., 0., 1., 1., 1., 0., 0.], requires_grad=True), tensor([0., 0., 0., 0., 0., 0., 1., 1.], requires_grad=True))
(tensor([0., 0., 0., 0., 0., 1., 0., 0.], requires_grad=True), tensor([0., 0., 0., 0., 0., 0., 0., 1.], requires_grad=True))
(tensor([0., 0., 1., 1., 0., 0., 1., 0.], requires_grad=True), tensor([0., 0., 0., 0., 0., 0., 1., 1.], requires_grad=True))
(tensor([0., 0., 0., 1., 0., 0., 1., 1.], requires_grad=True), tensor([0., 0., 0., 0., 0., 0., 1., 1.], requires_grad=True))
(tensor([0., 0., 0., 0., 1., 0., 1., 1.], requires_grad=True), tensor([0., 0., 0., 0., 0., 0., 1., 1.], requires_grad=True))


In [107]:
num_layers = 2
input_dim = 8
hidden_dim = 32
output_dim = 8

lr = 1e-3

model = nn.Sequential(MLP(num_layers, input_dim, hidden_dim, output_dim), nn.Sigmoid())
opt = optim.SGD(model.parameters(), lr=lr)
loss_fn = F.binary_cross_entropy

# number of SAMPLES classified correctly, NOT the number of bits
def bitvector_classification_accuracy(y_pred, y):
    return (y_pred.round() == y).float().mean()

In [108]:
epochs = 100

model = th.compile(model)
model.train()
train(model, loss_fn, opt, train_loader, epochs=epochs, acy_fn=bitvector_classification_accuracy)

loss: 0.134, acy: 0.000: 100%|██████████| 100/100 [03:10<00:00,  1.90s/it]


In [126]:
th.set_printoptions(sci_mode=False)
model(x)

tensor([[    0.0001,     0.0000,     0.0000,     0.0001,     0.0000,     0.0446,
             0.7048,     0.4984],
        [    0.0015,     0.0018,     0.0015,     0.0016,     0.0015,     0.0002,
             0.6483,     0.5658],
        [    0.0001,     0.0001,     0.0001,     0.0001,     0.0001,     0.0457,
             0.6877,     0.5257],
        [    0.0001,     0.0001,     0.0001,     0.0001,     0.0001,     0.0544,
             0.6772,     0.4926],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0492,
             0.6754,     0.4870],
        [    0.0013,     0.0018,     0.0015,     0.0015,     0.0015,     0.0001,
             0.6857,     0.5657],
        [    0.0001,     0.0001,     0.0001,     0.0001,     0.0001,     0.0544,
             0.6772,     0.4926],
        [    0.0015,     0.0018,     0.0015,     0.0016,     0.0015,     0.0002,
             0.6483,     0.5658],
        [    0.0001,     0.0001,     0.0001,     0.0001,     0.0001,     0.0698,